In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

import warnings; warnings.simplefilter('ignore')

In [2]:
credits=pd.read_csv(r'E:\ML&AI\RECOMMENDER SYSTEMS\rec-data\credits.csv')
keywords=pd.read_csv(r'E:\ML&AI\RECOMMENDER SYSTEMS\rec-data\keywords.csv')
md = pd. read_csv(r'E:\ML&AI\RECOMMENDER SYSTEMS\rec-data\movies_metadata.csv')

In [3]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [4]:
credits.dtypes

cast    object
crew    object
id       int64
dtype: object

In [5]:
credits.info

<bound method DataFrame.info of                                                     cast  \
0      [{'cast_id': 14, 'character': 'Woody (voice)',...   
1      [{'cast_id': 1, 'character': 'Alan Parrish', '...   
2      [{'cast_id': 2, 'character': 'Max Goldman', 'c...   
3      [{'cast_id': 1, 'character': "Savannah 'Vannah...   
4      [{'cast_id': 1, 'character': 'George Banks', '...   
...                                                  ...   
45471  [{'cast_id': 0, 'character': '', 'credit_id': ...   
45472  [{'cast_id': 1002, 'character': 'Sister Angela...   
45473  [{'cast_id': 6, 'character': 'Emily Shaw', 'cr...   
45474  [{'cast_id': 2, 'character': '', 'credit_id': ...   
45475                                                 []   

                                                    crew      id  
0      [{'credit_id': '52fe4284c3a36847f8024f49', 'de...     862  
1      [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...    8844  
2      [{'credit_id': '52fe466a9251416c75077a8

In [6]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [7]:
keywords.dtypes

id           int64
keywords    object
dtype: object

In [8]:
keywords.info

<bound method DataFrame.info of            id                                           keywords
0         862  [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
1        8844  [{'id': 10090, 'name': 'board game'}, {'id': 1...
2       15602  [{'id': 1495, 'name': 'fishing'}, {'id': 12392...
3       31357  [{'id': 818, 'name': 'based on novel'}, {'id':...
4       11862  [{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...
...       ...                                                ...
46414  439050             [{'id': 10703, 'name': 'tragic love'}]
46415  111109  [{'id': 2679, 'name': 'artist'}, {'id': 14531,...
46416   67758                                                 []
46417  227506                                                 []
46418  461257                                                 []

[46419 rows x 2 columns]>

In [9]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')

In [10]:
md.shape

(45466, 24)

In [11]:
credits['cast'] = credits['cast'].apply(literal_eval)
credits['crew'] = credits['crew'].apply(literal_eval)

In [12]:
keywords['keywords'] = keywords['keywords'].apply(literal_eval)

In [13]:
credits.isna().sum()

cast    0
crew    0
id      0
dtype: int64

In [14]:
keywords.isna().sum()

id          0
keywords    0
dtype: int64

In [15]:
df = credits.merge(keywords, on ='id')

In [16]:
df.head()

,cast,crew,id,keywords
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [17]:
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [18]:
#taking the name of the director from all the crew members
#we will only use directors from the crew column for our purpose
df['crew'] = df['crew'].apply(lambda x: [i['name'].lower() for i in x if i['job']=='Director'])

In [19]:
#taking the cast and keywords from the list of dictionaries of those columns
df['cast'] = df['cast'].apply(lambda x: [i['name'].lower() for i in x])
df['keywords'] = df['keywords'].apply(lambda x: [i['name'].lower() for i in x])

In [20]:
df.head()

,cast,crew,id,keywords
0,"[tom hanks, tim allen, don rickles, jim varney...",[john lasseter],862,"[jealousy, toy, boy, friendship, friends, riva..."
1,"[robin williams, jonathan hyde, kirsten dunst,...",[joe johnston],8844,"[board game, disappearance, based on children'..."
2,"[walter matthau, jack lemmon, ann-margret, sop...",[howard deutch],15602,"[fishing, best friend, duringcreditsstinger, o..."
3,"[whitney houston, angela bassett, loretta devi...",[forest whitaker],31357,"[based on novel, interracial relationship, sin..."
4,"[steve martin, diane keaton, martin short, kim...",[charles shyer],11862,"[baby, midlife crisis, confidence, aging, daug..."


In [23]:
title = md[['original_title', 'id']]
title.dtypes

original_title    object
id                object
dtype: object

In [25]:
#clean movie_id function
def clean_id(x):
    try:
        return int(x)
    except:
        return np.nan
title['id'] =title['id'].apply(clean_id)
title = title[title['id'].notnull()]
title['id'] = title['id'].astype(int)

In [26]:
title.dtypes

original_title    object
id                 int32
dtype: object

In [27]:
df = df.merge(title, on = 'id')
df.head()

,cast,crew,id,keywords,original_title
0,"[tom hanks, tim allen, don rickles, jim varney...",[john lasseter],862,"[jealousy, toy, boy, friendship, friends, riva...",Toy Story
1,"[robin williams, jonathan hyde, kirsten dunst,...",[joe johnston],8844,"[board game, disappearance, based on children'...",Jumanji
2,"[walter matthau, jack lemmon, ann-margret, sop...",[howard deutch],15602,"[fishing, best friend, duringcreditsstinger, o...",Grumpier Old Men
3,"[whitney houston, angela bassett, loretta devi...",[forest whitaker],31357,"[based on novel, interracial relationship, sin...",Waiting to Exhale
4,"[steve martin, diane keaton, martin short, kim...",[charles shyer],11862,"[baby, midlife crisis, confidence, aging, daug...",Father of the Bride Part II


In [28]:
#remove spaces the columns
df['cast'] = df['cast'].apply(lambda x: [i.replace(' ','') for i in x])
df['crew'] = df['crew'].apply(lambda x: [i.replace(' ','') for i in x])
df['keywords'] = df['keywords'].apply(lambda x: [i.replace(' ','') for i in x])

In [29]:
df['metadata'] = df.apply(lambda x : ' '.join(x['cast']) + ' ' + ' '.join(x['crew']) + ' ' + ' '.join(x['keywords']), axis = 1)

In [30]:
df.head()

,cast,crew,id,keywords,original_title,metadata
0,"[tomhanks, timallen, donrickles, jimvarney, wa...",[johnlasseter],862,"[jealousy, toy, boy, friendship, friends, riva...",Toy Story,tomhanks timallen donrickles jimvarney wallace...
1,"[robinwilliams, jonathanhyde, kirstendunst, br...",[joejohnston],8844,"[boardgame, disappearance, basedonchildren'sbo...",Jumanji,robinwilliams jonathanhyde kirstendunst bradle...
2,"[waltermatthau, jacklemmon, ann-margret, sophi...",[howarddeutch],15602,"[fishing, bestfriend, duringcreditsstinger, ol...",Grumpier Old Men,waltermatthau jacklemmon ann-margret sophialor...
3,"[whitneyhouston, angelabassett, lorettadevine,...",[forestwhitaker],31357,"[basedonnovel, interracialrelationship, single...",Waiting to Exhale,whitneyhouston angelabassett lorettadevine lel...
4,"[stevemartin, dianekeaton, martinshort, kimber...",[charlesshyer],11862,"[baby, midlifecrisis, confidence, aging, daugh...",Father of the Bride Part II,stevemartin dianekeaton martinshort kimberlywi...


In [31]:
df.shape

(46628, 6)

In [33]:
df.to_csv(r'E:\ML&AI\RECOMMENDER SYSTEMS\metadata.csv')

In [34]:
df=pd.read_csv(r'E:\ML&AI\RECOMMENDER SYSTEMS\metadata.csv')

In [35]:
df

,Unnamed: 0,cast,crew,id,keywords,original_title,metadata
0,0,"['tomhanks', 'timallen', 'donrickles', 'jimvar...",['johnlasseter'],862,"['jealousy', 'toy', 'boy', 'friendship', 'frie...",Toy Story,tomhanks timallen donrickles jimvarney wallace...
1,1,"['robinwilliams', 'jonathanhyde', 'kirstenduns...",['joejohnston'],8844,"['boardgame', 'disappearance', ""basedonchildre...",Jumanji,robinwilliams jonathanhyde kirstendunst bradle...
2,2,"['waltermatthau', 'jacklemmon', 'ann-margret',...",['howarddeutch'],15602,"['fishing', 'bestfriend', 'duringcreditsstinge...",Grumpier Old Men,waltermatthau jacklemmon ann-margret sophialor...
3,3,"['whitneyhouston', 'angelabassett', 'lorettade...",['forestwhitaker'],31357,"['basedonnovel', 'interracialrelationship', 's...",Waiting to Exhale,whitneyhouston angelabassett lorettadevine lel...
4,4,"['stevemartin', 'dianekeaton', 'martinshort', ...",['charlesshyer'],11862,"['baby', 'midlifecrisis', 'confidence', 'aging...",Father of the Bride Part II,stevemartin dianekeaton martinshort kimberlywi...
...,...,...,...,...,...,...,...
46623,46623,"['leilahatami', 'kouroshtahami', 'elhamkorda']",['hamidnematollah'],439050,['tragiclove'],رگ خواب,leilahatami kouroshtahami elhamkorda hamidnema...
46624,46624,"['angelaquino', 'perrydizon', 'hazelorencio', ...",['lavdiaz'],111109,"['artist', 'play', 'pinoy']",Siglo ng Pagluluwal,angelaquino perrydizon hazelorencio joeltorre ...
46625,46625,"['erikaeleniak', 'adambaldwin', 'juliedupage',...",['markl.lester'],67758,[],Betrayal,erikaeleniak adambaldwin juliedupage jamesrema...
46626,46626,"['iwanmosschuchin', 'nathalielissenko', 'pavel...",['yakovprotazanov'],227506,[],Satana likuyushchiy,iwanmosschuchin nathalielissenko pavelpavlov a...


In [36]:
dfs = df.iloc[:10000,:] #due to memory issues, using first 10k rows
from pandas.core.describe import DataFrameDescriber
#DataFrameDescriber = dfs.sample(frac =.2)
count_vec = CountVectorizer(stop_words='english')
count_vec_matrix = count_vec.fit_transform(dfs['metadata'])
cosine_sim_matrix = cosine_similarity(count_vec_matrix, count_vec_matrix)
#movies index mapping
mapping = pd.Series(dfs.index,index = dfs['original_title'])

In [37]:
#recommender function to recommend movies based on metadata
def recommend_movies_based_on_metadata(movie_input):
  movie_index = mapping[movie_input]
 #get similarity values with other movies
  similarity_score = list(enumerate(cosine_sim_matrix[movie_index]))
  similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
 # Get the scores of the 15 most similar movies. Ignore the first movie.
  similarity_score = similarity_score[1:15]
  movie_indices = [i[0] for i in similarity_score]
  return (df['original_title'].iloc[movie_indices])

In [38]:
recommend_movies_based_on_metadata('Grumpier Old Men')

3357             Grumpy Old Men
1750          The Odd Couple II
5632                Buddy Buddy
7297        The Whole Ten Yards
3077           Carnal Knowledge
7336             Viva Las Vegas
1266     Some Kind of Wonderful
756              The Grass Harp
4422         The Great Outdoors
1529                 Out to Sea
1995                    Newsies
4060    3000 Miles to Graceland
3413             The Odd Couple
4836        The Cheap Detective
Name: original_title, dtype: object

In [39]:
recommend_movies_based_on_metadata('Jumanji')

9001                  Neco z Alenky
3418                   Return to Me
1339                 101 Dalmatians
3206    With Byrd at the South Pole
5046                        Wendigo
5479            Heartbreak Hospital
6869             The Cat in the Hat
1579                  The Ice Storm
651       James and the Giant Peach
764                 Pharaoh's  Army
3362                      Buddy Boy
631                      Diabolique
4698                Little Man Tate
1766                 Kissing a Fool
Name: original_title, dtype: object

In [40]:
recommend_movies_based_on_metadata('Toy Story')

3024               Toy Story 2
25                     Othello
1792         Quest for Camelot
215           Boys on the Side
1813    Dream for an Insomniac
4514             Harlem Nights
4656                         O
5559        Ernest In The Army
1902              Child's Play
3413            The Odd Couple
2644              Jules et Jim
4210               Startup.com
5721        The Santa Clause 2
4799                      Tape
Name: original_title, dtype: object